# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,261.988468,0.817808,1234,262.806276,261.170660,525.612551,522.341319
6,650.387583,-0.267085,1234,650.120498,650.654668,1300.240997,1301.309336
7,831.586588,-0.118991,1234,831.467597,831.705579,1662.935193,1663.411158
8,301.620592,0.387817,1234,302.008408,301.232775,604.016816,602.465550
9,43.450387,-0.931396,1234,42.518992,44.381783,85.037984,88.763566
10,105.366749,0.005065,1234,105.371814,105.361685,210.743629,210.723369
11,758.098497,0.100223,1234,758.198720,757.998274,1516.397440,1515.996549
12,668.213435,0.570631,1234,668.784066,667.642804,1337.568131,1335.285607
13,618.313065,0.982414,1234,619.295479,617.330651,1238.590957,1234.661303
14,855.411413,-0.820593,1234,854.590820,856.232007,1709.181640,1712.464014


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-262.806276,261.170660
6,-650.120498,650.654668
7,-831.467597,831.705579
8,-302.008408,301.232775
9,-42.518992,44.381783
10,-105.371814,105.361685
11,-758.198720,757.998274
12,-668.784066,667.642804
13,-619.295479,617.330651
14,-854.590820,856.232007


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-262.806276,261.170660
6,-650.120498,650.654668
7,-831.467597,831.705579
8,-302.008408,301.232775
9,-42.518992,44.381783
10,-105.371814,105.361685
11,-758.198720,757.998274
12,-668.784066,667.642804
13,-619.295479,617.330651
14,-854.590820,856.232007


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-262.80627553,  261.17065959],
       [-650.12049844,  650.65466819],
       [-831.4675966 ,  831.7055792 ],
       [-302.00840824,  301.23277477],
       [ -42.51899177,   44.38178284],
       [-105.37181449,  105.3616845 ],
       [-758.19871979,  757.99827443],
       [-668.78406573,  667.6428037 ],
       [-619.29547855,  617.33065148],
       [-854.59082012,  856.2320068 ]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,523.612551,-522.341319
6,1298.240997,-1301.309336
7,1660.935193,-1663.411158
8,602.016816,-602.465550
9,83.037984,-88.763566
10,208.743629,-210.723369
11,1514.397440,-1515.996549
12,1335.568131,-1335.285607
13,1236.590957,-1234.661303
14,1707.181640,-1712.464014


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
